# white and black

In [5]:
# 导入库
import cv2
import mediapipe as mp
import numpy as np

#读取视频信息
cap = cv2.VideoCapture(0)

#如果读取成功则输出视频
while True:
    ret, img = cap.read()
    if not ret:
        break;
    
    # 转换成蓝色
    img = cv2.flip(img, 1)
    
    
    #加上黑白
    Grayimg = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, img_bw = cv2.threshold(Grayimg, 105, 255,cv2.THRESH_BINARY) # 这里的第二个参数要调，是阈值！！
    
    cv2.imshow('img',img_bw)
    
    #退出按钮
    if cv2.waitKey(1)== ord('q'):
        print("Quit successfully")
        break

#好习惯


# 释放内存
cv2.destroyAllWindows()
# 释放摄像头
cap.release()

Quit successfully


# Change the picture to white, black and gray

In [6]:
# 导入库
import cv2
import mediapipe as mp
import numpy as np

#读取视频信息
cap = cv2.VideoCapture(0)

#如果读取成功则输出视频
while True:
    ret, img = cap.read()
    if not ret:
        break;
    
    # 转换成蓝色
    img = cv2.flip(img, 1)
    
    
    #加上黑白
    Grayimg = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, img_bw = cv2.threshold(Grayimg, 80, 255,cv2.THRESH_BINARY) # 这里的第二个参数要调，是阈值！！
    
    #取反
    inv=255-Grayimg
    #高斯模糊
    sigma=60
    ksize=15
    blur = cv2.GaussianBlur(inv, ksize=(ksize, ksize), sigmaX=sigma, sigmaY=sigma)
    
    #变成素描（黑白）
    res = cv2.divide(Grayimg, 255 - blur, scale=255)+8
    
    #翻转(白黑)
    res2= 255 - res
    
    #加上阴影
    res3 = cv2.addWeighted(res2,0.6,img_bw,0.4,1)
    
    cv2.imshow('img',res3)
    
    #退出按钮
    if cv2.waitKey(1)== ord('q'):
        print("Quit successfully")
        break

#好习惯

# 释放内存
cv2.destroyAllWindows()
# 释放摄像头
cap.release()

Quit successfully


# 画风调整

In [7]:
def changeImg(img):
    
    # 镜像翻转
    img = cv2.flip(img, 1)
    
    # 复制图像
    tmp = img.copy()
    
    #得到黑白两色
    
    #加上黑白
    Grayimg = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, img_bw = cv2.threshold(Grayimg, 80, 255,cv2.THRESH_BINARY) # 这里的第二个参数要调，是阈值！！
    
    #取反
    inv=255-Grayimg
    #高斯模糊
    sigma=60
    ksize=15
    blur = cv2.GaussianBlur(inv, ksize=(ksize, ksize), sigmaX=sigma, sigmaY=sigma)
    #变成素描
    res = cv2.divide(Grayimg, 255 - blur, scale=255)+8
    
    #翻转
    res= 255 - res
    
    #加上阴影
    img = cv2.addWeighted(res,0.6,img_bw,0.4,1)
    
    
#     lower = np.uint8([100,100,100])
#     upper = np.uint8([255,255,255])
    
#     white_mask = cv2.inRange(tmp,lower,upper)
#     masked = cv2.bitwise_and(tmp,tmp, mask = white_mask)
    
#     img = cv2.cvtColor(masked, cv2.COLOR_BGR2GRAY)
    
    #将灰度图转为BGR
    tmp[:, :, 1] = img*0.4
    tmp[:, :, 2] = img*0.4
    tmp[:, :, 0] = img*0.4+100
    
    #调整亮度和对比度
    img_t = cv2.cvtColor(tmp,cv2.COLOR_BGR2HSV)
    h,s,v = cv2.split(img_t)
    
    # 增加图像亮度
    v1 = np.clip(cv2.add(1*v,40),0,255)
    img1 = np.uint8(cv2.merge((h,s,v1)))
    img1 = cv2.cvtColor(img1,cv2.COLOR_HSV2BGR)
    
    return img1


In [8]:
# 导入库
import cv2
import mediapipe as mp
import numpy as np

#读取视频信息
cap = cv2.VideoCapture(0)

#如果读取成功则输出视频
while True:
    ret, img = cap.read()
    if not ret:
        break;
    
    cv2.imshow('img',changeImg(img))
    
    if cv2.waitKey(1)== ord('q'):
        print("Quit successfully")
        break

#好习惯
# 释放内存
cv2.destroyAllWindows()
# 释放摄像头
cap.release()

Quit successfully


# 按键暂停

In [88]:
# 导入库
import cv2
import pygame
import time
import mediapipe as mp

pygame.init()
# 摄像头
cap=cv2.VideoCapture(0)

# 导入由于根据坐标画点的函数
mpDraw = mp.solutions.drawing_utils

# 导入模型
mp_holistic = mp.solutions.holistic
holistic = mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5)

# 导入脸部绘制格式
faceLmsStyle = mpDraw.DrawingSpec(color=(255, 0, 0), thickness=1)

# 初始照片的序号
num=1

# 导入音频
sound = pygame.mixer.Sound('piece.mp3')

update = True

# 检测摄像头是否在开启状态
while(cap.isOpened()):
    # 按键判断
    k = cv2.waitKey(1) & 0xFF
    
    # 得到每帧的图像
    if(update):
        ret_flag,Vshow = cap.read()
        # 翻转一下
        imageBGR = cv2.flip(Vshow, 1)
        
        # 将其由BGR转为RGB
        image = cv2.cvtColor(imageBGR, cv2.COLOR_BGR2RGB)
        # 找到其他部分
        results = holistic.process(image)
    
        if results.face_landmarks:
            #print(results.face_landmarks.landmark)
            
            # 找到框框的屏幕长宽
            imgHeight = imageBGR.shape[0]
            imgWidth = imageBGR.shape[1]
            
            # 导入面部
            mpDraw.draw_landmarks(imageBGR, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS,faceLmsStyle)
            
            #遍历的时候显示是第几个点,找到点的位置
            for i, lm in enumerate(results.face_landmarks.landmark):
                ###################################
                #获得坐标后转成int好看些
                xPos = int(lm.x * imgWidth)
                yPos = int(lm.y * imgHeight)
                #将数字插入到图片里
                # (图片)，字符串，位置，字体，大小，颜色,粗度
                cv2.putText(imageBGR,str(i),(xPos-25,yPos+5),cv2.FONT_HERSHEY_SIMPLEX,0.4,(0,0,255),1)
        
        
        
        # 给予不更新指令
        if k == ord('s'):
            print("You push the s")
            sound.play()
            T1 = time.perf_counter()
            T2 = T1
            # 这里可以生成图片

            print("------------------------")
    
    #判断s拉到外边贼有艺术效果
    
    
    # 显示图像
    cv2.imshow("Capture_Test",imageBGR)
        
    # 退出程序
    if k == ord(' '):
        print("exit successfully")
        break
    
    T2 = time.perf_counter()
    if((T2 - T1)*1000<14500):
        update = False
    else:
        update = True

# 释放内存
cv2.destroyAllWindows()
# 释放摄像头
cap.release()

exit successfully


嘴部标号
左侧标号
右侧标号

#  张嘴暂停

In [89]:
import math
def dis(lm1,lm2):
    return math.pow(math.pow((lm1.x - lm2.x),2) + math.pow((lm1.y-lm2.y),2),0.5)

# 导入库
import cv2
import pygame
import time
import mediapipe as mp

pygame.init()
# 摄像头
cap=cv2.VideoCapture(0)

# 导入由于根据坐标画点的函数
mpDraw = mp.solutions.drawing_utils

# 导入模型
mp_holistic = mp.solutions.holistic
holistic = mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5)

# 导入脸部绘制格式
faceLmsStyle = mpDraw.DrawingSpec(color=(255, 0, 0), thickness=1)

# 初始照片的序号
num=1

# 导入音频
sound = pygame.mixer.Sound('piece.mp3')

#初始化
T1 = time.perf_counter() - 40
update = True

# 检测摄像头是否在开启状态
while(cap.isOpened()):
    # 按键判断
    k = cv2.waitKey(1) & 0xFF
    
    # 得到每帧的图像
    if(update):
        ret_flag,Vshow = cap.read()
        # 翻转一下
        imageBGR = cv2.flip(Vshow, 1)
        
        # 将其由BGR转为RGB
        image = cv2.cvtColor(imageBGR, cv2.COLOR_BGR2RGB)
        # 找到其他部分
        results = holistic.process(image)
    
        if results.face_landmarks:
            #print(results.face_landmarks.landmark)
            
            # 找到框框的屏幕长宽
            imgHeight = imageBGR.shape[0]
            imgWidth = imageBGR.shape[1]
            
            # 导入面部
            mpDraw.draw_landmarks(imageBGR, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS,faceLmsStyle)

            #遍历的时候显示是第几个点,找到点的位置
            for i, lm in enumerate(results.face_landmarks.landmark):
                ###################################
                #获得坐标后转成int好看些
                xPos = int(lm.x * imgWidth)
                yPos = int(lm.y * imgHeight)
                #将数字插入到图片里
                # (图片)，字符串，位置，字体，大小，颜色,粗度
                #cv2.putText(imageBGR,str(i),(xPos-25,yPos+5),cv2.FONT_HERSHEY_SIMPLEX,0.4,(0,0,255),1)
                #找到嘴的位置
                if i == 87 or i==317 or i==82: 
                    cv2.circle(imageBGR,(xPos,yPos),2,(166,56,56),cv2.FILLED)
            
            #检测张嘴
            rate = dis(results.face_landmarks.landmark[87],results.face_landmarks.landmark[317])/dis(results.face_landmarks.landmark[87],results.face_landmarks.landmark[82])
            # 给予不更新指令
            if(rate<0.5):
                print("Detect mouth open")
                sound.play()
                T1 = time.perf_counter()
                T2 = T1
                # 这里可以生成图片
                print("------------------------")
            
    #判断s拉到外边贼有艺术效果
    
    
    # 显示图像
    cv2.imshow("Capture_Test",imageBGR)
        
    # 退出程序
    if k == ord(' '):
        print("exit successfully")
        break
    
    T2 = time.perf_counter()
    if((T2 - T1)*1000<14500):
        update = False
    else:
        update = True

# 释放内存
cv2.destroyAllWindows()
# 释放摄像头
cap.release()

Detect mouth open
------------------------
exit successfully


# 直接加字体

In [90]:
import math
def dis(lm1,lm2):
    return math.pow(math.pow((lm1.x - lm2.x),2) + math.pow((lm1.y-lm2.y),2),0.5)

# 导入库
import cv2
import pygame
import time
import mediapipe as mp

pygame.init()
# 摄像头
cap=cv2.VideoCapture(0)

# 导入由于根据坐标画点的函数
mpDraw = mp.solutions.drawing_utils

# 导入模型
mp_holistic = mp.solutions.holistic
holistic = mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5)

# 导入脸部绘制格式
faceLmsStyle = mpDraw.DrawingSpec(color=(255, 0, 0), thickness=1)

# 初始照片的序号
num=1

# 导入音频
sound = pygame.mixer.Sound('piece.mp3')

#初始化
T1 = time.perf_counter() - 40
update = True

# 检测摄像头是否在开启状态
while(cap.isOpened()):
    # 按键判断
    k = cv2.waitKey(1) & 0xFF
    
    # 得到每帧的图像
    if(update):
        ret_flag,Vshow = cap.read()
        # 翻转一下
        imageBGR = cv2.flip(Vshow, 1)
        
        # 将其由BGR转为RGB
        image = cv2.cvtColor(imageBGR, cv2.COLOR_BGR2RGB)
        # 找到其他部分
        results = holistic.process(image)
    
        if results.face_landmarks:
            #print(results.face_landmarks.landmark)
            
            # 找到框框的屏幕长宽
            imgHeight = imageBGR.shape[0]
            imgWidth = imageBGR.shape[1]
            
            # 导入面部
            #mpDraw.draw_landmarks(imageBGR, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS,faceLmsStyle)


            # 看哪边长放在哪边
            a = results.face_landmarks.landmark[132].x
            b = 1 - results.face_landmarks.landmark[361].x
            if a > b:
                left = True
            else:
                left = False
            
            #根据左右来放置英文单词
            if(left):
                # English
                Eng_xPos = int(results.face_landmarks.landmark[132].x * imgWidth/2)
                Eng_yPos = int((1+results.face_landmarks.landmark[132].y)/2 * imgHeight)
                # Chinese
                qun_xPos = Eng_xPos
                qun_yPos = int(results.face_landmarks.landmark[127].y* imgHeight)
                qing_xPos = Eng_xPos
                qing_yPos = int(results.face_landmarks.landmark[127].y* imgHeight+50)
            else:
                # English
                Eng_xPos = int((1+results.face_landmarks.landmark[361].x) * imgWidth/2)
                Eng_yPos = int((1+results.face_landmarks.landmark[361].y)/2 * imgHeight)
                # Chinese
                qun_xPos = Eng_xPos
                qun_yPos = int(results.face_landmarks.landmark[454].y* imgHeight)
                qing_xPos = Eng_xPos
                qing_yPos = int(results.face_landmarks.landmark[454].y* imgHeight+50)
                
            cv2.putText(imageBGR,"YOASOBI",(Eng_xPos-50,Eng_yPos),cv2.FONT_HERSHEY_PLAIN,2,(255,255,255),2)
            cv2.putText(imageBGR,"Q",(qun_xPos+40,qun_yPos),cv2.FONT_HERSHEY_SIMPLEX,2,(255,255,255),2)
            cv2.putText(imageBGR,"q",(qing_xPos+40,qing_yPos),cv2.FONT_HERSHEY_SIMPLEX,2,(255,255,255),2)
                
            #遍历的时候显示是第几个点,找到点的位置
            for i, lm in enumerate(results.face_landmarks.landmark):
                ###################################
                #获得坐标后转成int好看些
                xPos = int(lm.x * imgWidth)
                yPos = int(lm.y * imgHeight)
                
                # 将数字插入到图片里
                
                # (图片)，字符串，位置，字体，大小，颜色,粗度
                #cv2.putText(imageBGR,str(i),(xPos-25,yPos+5),cv2.FONT_HERSHEY_SIMPLEX,0.4,(0,0,255),1)
                
                # 找到耳朵对应的位置
                if i == 162 or i == 389: 
                    cv2.circle(imageBGR,(xPos,yPos),2,(166,56,56),cv2.FILLED)
                
            #检测张嘴
            rate = dis(results.face_landmarks.landmark[87],results.face_landmarks.landmark[317])/dis(results.face_landmarks.landmark[87],results.face_landmarks.landmark[82])
            
            # 给予不更新指令
            if(rate<0.5):
                print("Detect mouth open")
                sound.play()
                T1 = time.perf_counter()
                T2 = T1
                # 这里可以生成图片
                print("------------------------")
            
    #判断s拉到外边贼有艺术效果
    
    
    # 显示图像
    cv2.imshow("Capture_Test",imageBGR)
        
    # 退出程序
    if k == ord(' '):
        print("exit successfully")
        break
    
    T2 = time.perf_counter()
    if((T2 - T1)*1000<14500):
        update = False
    else:
        update = True

# 释放内存
cv2.destroyAllWindows()
# 释放摄像头
cap.release()

exit successfully


# 张嘴加上字体

In [91]:
import math
def dis(lm1,lm2):
    return math.pow(math.pow((lm1.x - lm2.x),2) + math.pow((lm1.y-lm2.y),2),0.5)

# 导入库
import cv2
import pygame
import time
import mediapipe as mp

pygame.init()
# 摄像头
cap=cv2.VideoCapture(0)

# 导入由于根据坐标画点的函数
mpDraw = mp.solutions.drawing_utils

# 导入模型
mp_holistic = mp.solutions.holistic
holistic = mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5)

# 导入脸部绘制格式
faceLmsStyle = mpDraw.DrawingSpec(color=(255, 0, 0), thickness=1)

# 初始照片的序号
num=1

# 导入音频
sound = pygame.mixer.Sound('piece.mp3')

#初始化
T1 = time.perf_counter() - 40
update = True

# 检测摄像头是否在开启状态
while(cap.isOpened()):
    # 按键判断
    k = cv2.waitKey(1) & 0xFF
    
    # 得到每帧的图像
    if(update):
        ret_flag,Vshow = cap.read()
        # 翻转一下
        imageBGR = cv2.flip(Vshow, 1)
        
        # 将其由BGR转为RGB
        image = cv2.cvtColor(imageBGR, cv2.COLOR_BGR2RGB)
        # 找到其他部分
        results = holistic.process(image)
    
        if results.face_landmarks:
            #print(results.face_landmarks.landmark)
            
            #检测张嘴
            rate = dis(results.face_landmarks.landmark[87],results.face_landmarks.landmark[317])/dis(results.face_landmarks.landmark[87],results.face_landmarks.landmark[82])
            
            # 给予不更新指令
            if(rate<0.5):
                print("Detect mouth open")
                sound.play()
                T1 = time.perf_counter()
                T2 = T1
                
                # 这里可以生成图片
                
                # 找到框框的屏幕长宽
                imgHeight = imageBGR.shape[0]
                imgWidth = imageBGR.shape[1]
            
                # 导入面部
                #mpDraw.draw_landmarks(imageBGR, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS,faceLmsStyle)


                # 看哪边长放在哪边
                a = results.face_landmarks.landmark[132].x
                b = 1 - results.face_landmarks.landmark[361].x
                if a > b:
                    left = True
                else:
                    left = False
            
                #根据左右来放置英文单词
                if(left):
                    # English
                    Eng_xPos = int(results.face_landmarks.landmark[132].x * imgWidth/2)
                    Eng_yPos = int((1+results.face_landmarks.landmark[132].y)/2 * imgHeight)
                    # Chinese
                    qun_xPos = Eng_xPos
                    qun_yPos = int(results.face_landmarks.landmark[127].y* imgHeight)
                    qing_xPos = Eng_xPos
                    qing_yPos = int(results.face_landmarks.landmark[127].y* imgHeight+50)
                else:
                    # English
                    Eng_xPos = int((1+results.face_landmarks.landmark[361].x) * imgWidth/2)
                    Eng_yPos = int((1+results.face_landmarks.landmark[361].y)/2 * imgHeight)
                    # Chinese
                    qun_xPos = Eng_xPos
                    qun_yPos = int(results.face_landmarks.landmark[454].y* imgHeight)
                    qing_xPos = Eng_xPos
                    qing_yPos = int(results.face_landmarks.landmark[454].y* imgHeight+50)
                
                cv2.putText(imageBGR,"YOASOBI",(Eng_xPos-50,Eng_yPos),cv2.FONT_HERSHEY_PLAIN,2,(255,255,255),2)
                cv2.putText(imageBGR,"Q",(qun_xPos+40,qun_yPos),cv2.FONT_HERSHEY_SIMPLEX,2,(255,255,255),2)
                cv2.putText(imageBGR,"q",(qing_xPos+40,qing_yPos),cv2.FONT_HERSHEY_SIMPLEX,2,(255,255,255),2)
                
                '''
                #遍历的时候显示是第几个点,找到点的位置
                for i, lm in enumerate(results.face_landmarks.landmark):
                    ###################################
                    #获得坐标后转成int好看些
                    xPos = int(lm.x * imgWidth)
                    yPos = int(lm.y * imgHeight)

                    # 将数字插入到图片里

                    # (图片)，字符串，位置，字体，大小，颜色,粗度
                    #cv2.putText(imageBGR,str(i),(xPos-25,yPos+5),cv2.FONT_HERSHEY_SIMPLEX,0.4,(0,0,255),1)

                    # 找到耳朵对应的位置
                    if i == 162 or i == 389: 
                        cv2.circle(imageBGR,(xPos,yPos),2,(166,56,56),cv2.FILLED)
                '''
                
                print("------------------------")
            
    #判断s拉到外边贼有艺术效果
    
    
    # 显示图像
    cv2.imshow("Capture_Test",imageBGR)
        
    # 退出程序
    if k == ord(' '):
        print("exit successfully")
        break
    
    T2 = time.perf_counter()
    if((T2 - T1)*1000<14500):
        update = False
    else:
        update = True

# 释放内存
cv2.destroyAllWindows()
# 释放摄像头
cap.release()

Detect mouth open
------------------------
exit successfully


# 加上画风

In [2]:
def changeImg(img):
    
    # 复制图像
    tmp = img.copy()
    
    # 灰度化以后二值化
    Grayimg = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, img = cv2.threshold(Grayimg, 110, 255,cv2.THRESH_BINARY) # 这里的第二个参数要调，是阈值！！
    #print(b)
    # set green and red channels to 0
    
    #将灰度图转为BGR
    tmp[:, :, 1] = img*0.3
    tmp[:, :, 2] = img*0.3
    tmp[:, :, 0] = img*0.3+130
    
    #调整亮度和对比度
    img_t = cv2.cvtColor(tmp,cv2.COLOR_BGR2HSV)
    h,s,v = cv2.split(img_t)
    
    # 增加图像亮度
    v1 = np.clip(cv2.add(1*v,40),0,255)
    img1 = np.uint8(cv2.merge((h,s,v1)))
    img1 = cv2.cvtColor(img1,cv2.COLOR_HSV2BGR)
    
    return img1


In [3]:
import math
def dis(lm1,lm2):
    return math.pow(math.pow((lm1.x - lm2.x),2) + math.pow((lm1.y-lm2.y),2),0.5)

# 导入库
import cv2
import pygame
import time
import mediapipe as mp

pygame.init()
# 摄像头
cap=cv2.VideoCapture(0)

# 导入由于根据坐标画点的函数
mpDraw = mp.solutions.drawing_utils

# 导入模型
mp_holistic = mp.solutions.holistic
holistic = mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5)

# 导入脸部绘制格式
faceLmsStyle = mpDraw.DrawingSpec(color=(255, 0, 0), thickness=1)

# 初始照片的序号
num=1

# 导入音频
sound = pygame.mixer.Sound('piece.mp3')

#初始化
T1 = time.perf_counter() - 40
update = True

# 检测摄像头是否在开启状态
while(cap.isOpened()):
    # 按键判断
    k = cv2.waitKey(1) & 0xFF
    
    # 得到每帧的图像
    if(update):
        ret_flag,Vshow = cap.read()
        # 翻转一下
        imageBGR = cv2.flip(Vshow, 1)
        
        # 将其由BGR转为RGB
        image = cv2.cvtColor(imageBGR, cv2.COLOR_BGR2RGB)
        # 找到其他部分
        results = holistic.process(image)
    
        if results.face_landmarks:
            #print(results.face_landmarks.landmark)
            
            #检测张嘴
            rate = dis(results.face_landmarks.landmark[87],results.face_landmarks.landmark[317])/dis(results.face_landmarks.landmark[87],results.face_landmarks.landmark[82])
            
            # 给予不更新指令
            if(rate<0.5):
                print("Detect mouth open")
                sound.play()
                T1 = time.perf_counter()
                T2 = T1
                
                # 这里可以生成图片
                
                # 找到框框的屏幕长宽
                imgHeight = imageBGR.shape[0]
                imgWidth = imageBGR.shape[1]
            
                # 导入面部
                #mpDraw.draw_landmarks(imageBGR, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS,faceLmsStyle)

                imageBGR=changeImg(imageBGR)
                
                # 看哪边长放在哪边
                a = results.face_landmarks.landmark[132].x
                b = 1 - results.face_landmarks.landmark[361].x
                if a > b:
                    left = True
                else:
                    left = False
                
                #根据左右来放置英文单词
                if(left):
                    # English
                    Eng_xPos = int(results.face_landmarks.landmark[132].x * imgWidth/2)
                    Eng_yPos = int((1+results.face_landmarks.landmark[132].y)/2 * imgHeight)
                    # Chinese
                    qun_xPos = Eng_xPos
                    qun_yPos = int(results.face_landmarks.landmark[127].y* imgHeight)
                    qing_xPos = Eng_xPos
                    qing_yPos = int(results.face_landmarks.landmark[127].y* imgHeight+50)
                else:
                    # English
                    Eng_xPos = int((1+results.face_landmarks.landmark[361].x) * imgWidth/2)
                    Eng_yPos = int((1+results.face_landmarks.landmark[361].y)/2 * imgHeight)
                    # Chinese
                    qun_xPos = Eng_xPos
                    qun_yPos = int(results.face_landmarks.landmark[454].y* imgHeight)
                    qing_xPos = Eng_xPos
                    qing_yPos = int(results.face_landmarks.landmark[454].y* imgHeight+50)
                
                cv2.putText(imageBGR,"YOASOBI",(Eng_xPos-50,Eng_yPos),cv2.FONT_HERSHEY_PLAIN,2,(255,255,255),2)
                cv2.putText(imageBGR,"Q",(qun_xPos+40,qun_yPos),cv2.FONT_HERSHEY_SIMPLEX,2,(255,255,255),2)
                cv2.putText(imageBGR,"q",(qing_xPos+40,qing_yPos),cv2.FONT_HERSHEY_SIMPLEX,2,(255,255,255),2)
                

                '''
                #遍历的时候显示是第几个点,找到点的位置
                for i, lm in enumerate(results.face_landmarks.landmark):
                    ###################################
                    #获得坐标后转成int好看些
                    xPos = int(lm.x * imgWidth)
                    yPos = int(lm.y * imgHeight)

                    # 将数字插入到图片里

                    # (图片)，字符串，位置，字体，大小，颜色,粗度
                    #cv2.putText(imageBGR,str(i),(xPos-25,yPos+5),cv2.FONT_HERSHEY_SIMPLEX,0.4,(0,0,255),1)

                    # 找到耳朵对应的位置
                    if i == 162 or i == 389: 
                        cv2.circle(imageBGR,(xPos,yPos),2,(166,56,56),cv2.FILLED)
                '''
                
                
                print("------------------------")
            
    #判断s拉到外边贼有艺术效果
    
    
    # 显示图像
    cv2.imshow("Capture_Test",imageBGR)
        
    # 退出程序
    if k == ord(' '):
        print("exit successfully")
        break
    
    T2 = time.perf_counter()
    if((T2 - T1)*1000<14500):
        update = False
    else:
        update = True

# 释放内存
cv2.destroyAllWindows()
# 释放摄像头
cap.release()

pygame 2.1.2 (SDL 2.0.18, Python 3.8.8)
Hello from the pygame community. https://www.pygame.org/contribute.html
exit successfully


# 改中文

In [1]:
def changeImg(img):
    
    # 复制图像
    tmp = img.copy()
    
    #得到黑白两色
    
    #加上黑白
    Grayimg = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, img_bw = cv2.threshold(Grayimg, 130, 255,cv2.THRESH_BINARY) # 这里的第二个参数要调，是阈值！！
    
    #取反
    inv=255-Grayimg
    #高斯模糊
    sigma=60
    ksize=15
    blur = cv2.GaussianBlur(inv, ksize=(ksize, ksize), sigmaX=sigma, sigmaY=sigma)
    #变成素描
    res = cv2.divide(Grayimg, 255 - blur, scale=255)+8
    
    #翻转
    res= 255 - res
    
    #加上阴影
    img = cv2.addWeighted(res,0.6,img_bw,0.4,1)
    
    
#     lower = np.uint8([100,100,100])
#     upper = np.uint8([255,255,255])
    
#     white_mask = cv2.inRange(tmp,lower,upper)
#     masked = cv2.bitwise_and(tmp,tmp, mask = white_mask)
    
#     img = cv2.cvtColor(masked, cv2.COLOR_BGR2GRAY)
    
    #将灰度图转为BGR
    tmp[:, :, 1] = img*0.4
    tmp[:, :, 2] = img*0.4
    tmp[:, :, 0] = img*0.4+100
    
    #调整亮度和对比度
    img_t = cv2.cvtColor(tmp,cv2.COLOR_BGR2HSV)
    h,s,v = cv2.split(img_t)
    
    # 增加图像亮度
    v1 = np.clip(cv2.add(1*v,40),0,255)
    img1 = np.uint8(cv2.merge((h,s,v1)))
    img1 = cv2.cvtColor(img1,cv2.COLOR_HSV2BGR)
    
    return img1

In [2]:
import numpy as np
import math
def changeImg_easy(img):
    
    # 复制图像
    tmp = img.copy()
    
    #将灰度图转为BGR
    tmp[:, :, 1] = 0
    tmp[:, :, 2] = 0
        
    return tmp


In [3]:
from PIL import Image, ImageDraw, ImageFont
def cv2AddChineseText(img, text, position,textSize=30,textColor=(0, 255, 0)):
    if (isinstance(img, np.ndarray)):  # 判断是否OpenCV图片类型
        img = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    # 创建一个可以在给定图像上绘图的对象
    draw = ImageDraw.Draw(img)
    # 字体的格式
    fontStyle = ImageFont.truetype("Alibaba-PuHuiTi-Medium.ttf", textSize, encoding="utf-8")
    # 绘制文本
    draw.text(position, text, textColor, font=fontStyle)
    # 转换回OpenCV格式
    return cv2.cvtColor(np.asarray(img), cv2.COLOR_RGB2BGR)

In [ ]:
import math
def dis(lm1,lm2):
    return math.pow(math.pow((lm1.x - lm2.x),2) + math.pow((lm1.y-lm2.y),2),0.5)

# 导入库
import cv2
import pygame
import time
import mediapipe as mp

pygame.init()
# 摄像头
cap=cv2.VideoCapture(0)

# 导入由于根据坐标画点的函数
mpDraw = mp.solutions.drawing_utils

# 导入模型
mp_holistic = mp.solutions.holistic
holistic = mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5)

# 导入脸部绘制格式
faceLmsStyle = mpDraw.DrawingSpec(color=(255, 0, 0), thickness=1)

# 初始照片的序号
num=1

# 导入音频
sound = pygame.mixer.Sound('piece.mp3')

#初始化
T1 = time.perf_counter() - 40
update = True

# 检测摄像头是否在开启状态
while(cap.isOpened()):
    # 按键判断
    k = cv2.waitKey(1) & 0xFF
    
    # 得到每帧的图像
    if(update):
        ret_flag,Vshow = cap.read()
        # 翻转一下
        imageBGR = cv2.flip(Vshow, 1)
        
        # 将其由BGR转为RGB
        image = cv2.cvtColor(imageBGR, cv2.COLOR_BGR2RGB)
        # 找到其他部分
        results = holistic.process(image)
    
        if results.face_landmarks:
            #print(results.face_landmarks.landmark)
            
            #检测张嘴
            rate = dis(results.face_landmarks.landmark[87],results.face_landmarks.landmark[317])/dis(results.face_landmarks.landmark[87],results.face_landmarks.landmark[82])
            
            # 给予不更新指令
            if(rate<0.5):
                print("Detect mouth open")
                sound.play()
                T1 = time.perf_counter()
                T2 = T1
                
                # 这里可以生成图片
                
                # 找到框框的屏幕长宽
                imgHeight = imageBGR.shape[0]
                imgWidth = imageBGR.shape[1]
            
                # 导入面部
                #mpDraw.draw_landmarks(imageBGR, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS,faceLmsStyle)

                imageBGR=changeImg(imageBGR)
                
                # 看哪边长放在哪边
                a = results.face_landmarks.landmark[132].x
                b = 1 - results.face_landmarks.landmark[361].x
                if a > b:
                    left = True
                else:
                    left = False
                
                #根据左右来放置英文单词
                if(left):
                    # English
                    Eng_xPos = int(results.face_landmarks.landmark[132].x * imgWidth/2)
                    Eng_yPos = int((1+results.face_landmarks.landmark[132].y)/2 * imgHeight)
                    # Chinese
                    qun_xPos = Eng_xPos
                    qun_yPos = int(results.face_landmarks.landmark[127].y* imgHeight)
                    qing_xPos = Eng_xPos
                    qing_yPos = int(results.face_landmarks.landmark[127].y* imgHeight+50)
                else:
                    # English
                    Eng_xPos = int((1+results.face_landmarks.landmark[361].x) * imgWidth/2)
                    Eng_yPos = int((1+results.face_landmarks.landmark[361].y)/2 * imgHeight)
                    # Chinese
                    qun_xPos = Eng_xPos
                    qun_yPos = int(results.face_landmarks.landmark[454].y * imgHeight)
                    qing_xPos = Eng_xPos
                    qing_yPos = int(results.face_landmarks.landmark[454].y * imgHeight+50)
                
                cv2.putText(imageBGR,"YOASOBI",(Eng_xPos-50,Eng_yPos),cv2.FONT_HERSHEY_PLAIN,2,(255,255,255),2)
                imageBGR=cv2AddChineseText(imageBGR,"群",(qun_xPos+40,qun_yPos),40,(255,255,255))
                imageBGR=cv2AddChineseText(imageBGR,"青",(qing_xPos+40,qing_yPos),40,(255,255,255))
                

                '''
                #遍历的时候显示是第几个点,找到点的位置
                for i, lm in enumerate(results.face_landmarks.landmark):
                    ###################################
                    #获得坐标后转成int好看些
                    xPos = int(lm.x * imgWidth)
                    yPos = int(lm.y * imgHeight)

                    # 将数字插入到图片里

                    # (图片)，字符串，位置，字体，大小，颜色,粗度
                    #cv2.putText(imageBGR,str(i),(xPos-25,yPos+5),cv2.FONT_HERSHEY_SIMPLEX,0.4,(0,0,255),1)

                    # 找到耳朵对应的位置
                    if i == 162 or i == 389: 
                        cv2.circle(imageBGR,(xPos,yPos),2,(166,56,56),cv2.FILLED)
                '''
                
                
                print("------------------------")
            
    #判断s拉到外边贼有艺术效果
    
    
    # 显示图像
    cv2.imshow("Capture_Test",imageBGR)
        
    # 退出程序
    if k == ord(' '):
        print("exit successfully")
        break
    
    T2 = time.perf_counter()
    if((T2 - T1)*1000<14500):
        update = False
    else:
        update = True

# 释放内存
cv2.destroyAllWindows()
# 释放摄像头
cap.release()

Detect mouth open
------------------------
